In [1]:
#Import the acrtuals to compare to model guesses
#nc file
import netCDF4
import numpy as np
import time
import matplotlib.pyplot as plt
import xarray as xr
import dask
print('begin')

#Path for Aqua_Planet, 2 months, No VPB
path_to_file = "/fast/gmooers/Real_Geography_Manuscript/Models/SPCAM3_6_Months/full_physics_essentials_valid_month02_targets.nc"
ds = xr.open_dataset(path_to_file)
truths = ds.targets[:, :30].values
lons = ds.lon.values
lats = ds.lat.values
print('halfway')
#Path for Aqua_Planet, 2 months, No VPB
path_to_file = "/fast/gmooers/Real_Geography_Manuscript/Models/SPCAM3_6_Months.nc"
ds = xr.open_dataset(path_to_file)
features = ds.Prediction[:, :30].values

#x = 144
x = 128
#y = 96
y=64
z = 30
t = int(len(truths)/(x*y))
print(t)

begin
halfway
3420


In [2]:
reconstructed_targets = np.zeros(shape=(x, y, t, z))
reconstructed_features = np.zeros(shape=(x, y, t, z))
#print(reconstructed_targets.shape)
print('hi')
count = 0
for i in range(t):
    for j in range(y):
        for k in range(x):
            A = truths[count]
            B = features[count]
            reconstructed_targets[k, j, i, :] = A
            reconstructed_features[k, j, i, :] = B
            count = count + 1

print(reconstructed_targets.shape)
print(reconstructed_features.shape)

hi
(144, 96, 3420, 30)
(144, 96, 3420, 30)


In [4]:
daily_predictions = np.empty(shape=(len(reconstructed_features), len(reconstructed_features[0]), int(len(reconstructed_features[0][0])/96), len(reconstructed_features[0][0][0])))
daily_truths = np.empty(shape=(len(reconstructed_targets), len(reconstructed_targets[0]), int(len(reconstructed_targets[0][0])/96), len(reconstructed_targets[0][0][0])))

In [ ]:
current = 0

for i in range(int(len(reconstructed_targets[0][0])/96)):
    daily_predictions[:,:,i,:] = np.nansum(reconstructed_features[:,:,current:current+96,:], axis=2)
    daily_truths[:,:,i,:] = np.nansum(reconstructed_targets[:,:,current:current+96,:],axis=2)
    current = current+96

In [ ]:
#np.isnan(daily_predictions).any()

In [ ]:
R = np.zeros(shape=(x, y, z))
SSE = np.zeros(shape=(x, y, z))
SVAR = np.zeros(shape=(x, y, z))


for i in range(len(daily_predictions)):
    for k in range(len(daily_predictions[0])):
        for j in range(len(daily_predictions[0][0][0])):
                target_array = np.squeeze(daily_truths[i,k,:,j])
                predict_array = np.squeeze(daily_predictions[i,k,:,j])
    
                sse = np.sum((target_array-predict_array)**2.0)
                svar = np.sum((target_array-np.mean(target_array))**2.0)
                r_2 = 1-(sse/svar)
                R[i, k, j] = r_2
                SSE[i, k, j] = sse
                SVAR[i, k, j] = svar
    
    if i%10==0:
        print(i)
        
R_Save = np.squeeze(R)
SSE_Save = np.squeeze(SSE)
SVAR_Save = np.squeeze(SVAR)

#np.save("/fast/gmooers/Other_R2/Improved_Daily_SSE_SPCAM5_15_Min_Interval_Heat.npy", SSE_Save)
#np.save("/fast/gmooers/Other_R2/Improved_Daily_SVAR_SPCAM5_15_Min_Interval_Heat.npy", SVAR_Save)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/SPCAM3_Daily_Interval_Heating.npy", R_Save)